In [27]:
from tracing.rl.actions import *
from tracing.rl.a3cmodel import A3CModel
from tracing.rl.rewards import PopupRewardsCalculator
from tracing.rl.environment import Environment
from tracing.rl.actor_learner import ActionsMemory
from tracing.rl.actor_learner import ActorLearnerWorker
import tensorflow as tf
import tensorflow.contrib.slim as slim
import threading
import json
import random
from scipy import misc
import numpy as np

from tracing.selenium_utils.controls import Types
from nltk.tokenize import word_tokenize

/home/aleksei/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
def split(arr, batch_size):
    batch = []
    for item in arr:
        if len(batch) >= batch_size:
            yield batch
            batch = []

        batch.append(item)

    if len(batch) > 0:
        yield batch
        
        
def read_img(ctrl, size = 300):
    img = misc.imread(ctrl['img_file'])
    img = (img - 128.)/128.
    shape = img.shape
    assert shape[0] == size or shape[1] == size, 'found image {}, control {}'.format(shape, ctrl)
    
    return img


In [32]:
class ControlTextModel:
    # If session is not defined than default session will be used
    def __init__(self, a3c_model, session, encoder, repr_dim = 200):
        self.word_repr = None
        self.word_embeddings = None
                
        self.a3c_model = a3c_model
        self.session = session 
        self.encoder = encoder
        self.repr_dim = repr_dim
        
        self.device = '/cpu:0'
        
        self.add_text_inputs()
        self.add_cnn_char_repr()
        self.add_pretrained_word_embeddings()
        self.add_context_repr(200)
        self.add_loss()
        self.add_training_op()
        
    
    def add_img_repr(self):
        with tf.device(self.device):
            self.img_repr = slim.fully_connected(self.a3c_model.net, self.repr_dim)
    
    
    def add_text_inputs(self):
        with tf.variable_scope("char_repr") as scope:
            # shape = (batch size, sentence, word)
            self.char_ids = tf.placeholder(tf.int32, shape=[None, None, None], name="char_ids")

            # shape = (batch_size, sentence)
            self.word_lengths = tf.placeholder(tf.int32, shape=[None, None], name="word_lengths")

        with tf.variable_scope("word_repr") as scope:
            # shape = (batch size)
            self.sentence_lengths = tf.placeholder(tf.int32, shape=[None], name="sentence_lengths")

        with tf.variable_scope("training", reuse=None) as scope:
            # shape = (batch, 2)
            # One hot labels if a3c_model.img is similar to current sentence
            self.similar = tf.placeholder(tf.float32, shape=[None, 2], name="similar")
            

            self.lr = tf.placeholder_with_default(0.005,  shape=(), name="lr")
            self.dropout = tf.placeholder_with_default(1., shape=(), name="dropout")
        
                
    def add_cnn_char_repr(self, nchars = 101, dim=25, nfilters=25, pad=2):
        with tf.device(self.device):
        
            with tf.variable_scope("char_repr_cnn") as scope:
                # 1. Lookup for character embeddings
                char_range = math.sqrt(3 / dim)
                embeddings = tf.get_variable(name="char_embeddings", dtype=tf.float32,
                    shape=[nchars, dim],
                    initializer=tf.random_uniform_initializer(-char_range, char_range))

                # shape = (batch, sentence, word_len, embeddings dim)
                char_embeddings = tf.nn.embedding_lookup(embeddings, self.char_ids)
                char_embeddings = tf.nn.dropout(char_embeddings, self.dropout)
                s = tf.shape(char_embeddings)

                # shape = (batch x sentence, word_len, embeddings dim)
                char_embeddings = tf.reshape(char_embeddings, shape=[-1, s[-2], dim])

                # batch x sentence, word_len, nfilters
                conv1d = tf.layers.conv1d(
                    char_embeddings,
                    filters=nfilters,
                    kernel_size=[3],
                    padding='same',
                    activation=tf.nn.relu
                )

                # Max across each filter, shape = (batch x sentence, nfilters)
                char_repr = tf.reduce_max(conv1d, axis=1, keep_dims=True)
                char_repr = tf.squeeze(char_repr, squeeze_dims=[1])

                # (batch, sentence, nfilters)
                char_repr = tf.reshape(char_repr, shape=[s[0], s[1], nfilters])

                if self.word_repr is not None:
                    self.word_repr = tf.concat([self.word_repr, char_rep], axis=-1)
                else:
                    self.word_repr = char_repr

    
    
    def add_pretrained_word_embeddings(self, dim=100, trainable=True):
        with tf.device(self.device):
            with tf.variable_scope("word_repr") as scope:
                # shape = (batch size, sentence, dim)
                self.word_embeddings = tf.placeholder(tf.float32, shape=[None, None, dim],
                                                      name="word_embeddings")

                if self.word_repr is not None:
                    self.word_repr = tf.concat([self.word_repr, self.word_embeddings], axis=-1)
                else:
                    self.word_repr = word_embeddings
    
    
    def extract_last(self, source, lengths):
        batch_range = tf.range(tf.shape(source)[0])
        batch_indices = tf.stack([batch_range, lengths - 1], axis=1)
        res = tf.gather_nd(source, batch_indices)

        return res
    
    
    # Adds LSTM with size of each cell hidden_size
    def add_context_repr(self, hidden_size=200):
        with tf.device(self.device):
        
            with tf.variable_scope("context_repr") as scope:
                cell = tf.contrib.rnn.LSTMCell(hidden_size)
                
                word_repr = tf.nn.dropout(self.word_repr, self.dropout)

                output, state = tf.nn.dynamic_rnn(
                    cell,
                    word_repr,
                    sequence_length=self.sentence_lengths,
                    dtype=tf.float32)

                context_repr = tf.nn.dropout(output, self.dropout)
                
                # batch x hidden_size
                sentence_repr = self.extract_last(context_repr, self.sentence_lengths)
                
#                 w_bound = math.sqrt(6 / (hidden_size))
#                 W = tf.get_variable("W", shape=[hidden_size, self.repr_dim],
#                                 dtype=tf.float32,
#                                 initializer=tf.random_uniform_initializer(-w_bound, w_bound))


#                 b = tf.get_variable("b", shape=[self.repr_dim], dtype=tf.float32)
                
                # batch x hidden_size
                self.text_repr = slim.fully_connected(sentence_repr, self.repr_dim)
    
    
    def add_loss(self):
        same = tf.losses.cosine_distance(self.text_repr, self.img_repr)
        not_same = 1 - same
        
        probas = tf.stack([same, not_same])
        
        sim_loss = self.similar * tf.log(probas)
        sim_loss = tf.reduce_sum(sim_loss, -1)
        
        self.text_loss = tf.reduce_mean(sim_loss)
    
    
    # clip_gradient < 0  - no gradient clipping
    def add_training_op(self, clip_gradient = 5.0):

        with tf.variable_scope("training", reuse=None) as scope:
            optimizer = tf.train.MomentumOptimizer(learning_rate=self.lr, momentum=0.9)
            if clip_gradient > 0:
                gvs = optimizer.compute_gradients(self.loss)
                capped_gvs = [(tf.clip_by_value(grad, -clip_gradient, clip_gradient), var) for 
                              grad, var in gvs]
                self.train_op = optimizer.apply_gradients(capped_gvs)
            else:
                self.train_op = optimizer.minimize(self.loss)

            self.init_op = tf.variables_initializer(tf.global_variables(), name="init")
    
    
    def get_control_text(self, ctrl):
        label = ctrl.get('label', '').strip()
        tip = ctrl.get('tip', '').strip()
        
        result = label if label != '' else tip
        return result.lower()
    
    
    def is_for_training(self, ctrl):
        txt = self.get_control_text(ctrl)
        return txt != ''
    
    
    def is_similar(self, ctrl1, ctrl2):
        text1 = self.get_control_text(ctrl1)
        text2 = self.get_control_text(ctrl2)
        
        tokens1 = set(word_tokenize(text1))
        tokens2 = set(word_tokenize(text2))
        intesection = len(tokens1.intesect(tokens2))
        
        scale = intesection / min(len(tokens1), len(tokens2))
                    
        return scale >= 1 / 2
    
    
    def extract_pairs(self, controls, neg_samples = 5):
        result = []
        for ctrl in controls:
            result.append((ctrl, ctrl, True))
            
            for i in range(neg_samples):
                for _ in range(5):
                    neg_ctrl = random.sample(controls)
                    if not self.is_similar(ctrl, neg_ctrl):
                        result.append(ctrl, neg_ctrl, False)
                        break                    
                    
        
        random.shuffle(result)
        return result
        
    
    def batch_to_input(self, batch):
        imgs = []
        texts = []
        similarities = []
        
        for ctrl, ctrl2, similar in batch:
            imgs.append(read_img(ctrl))
            texts.append(self.get_control_text(ctrl2))
            
            sim = [1, 0] if similar else [0, 1]
            similarities.append(sim)
        
        text_input = self.encoder.encode(texts)
        
        return {
            self.a3c_model.img: imgs,
            self.similar: similarities,
            
            self.char_ids: text_input['char_ids'],
            self.word_embeddings: text_input['word_embeddings'],
            self.sentence_lengths: text_input['sentence_lengths'],
            selt.word_lengths: text_input['word_lengths']
        }
        
        
    
    def train(self, controls, word_embeddings, epochs = 5, neg_samples = 5):
        to_train = list(filter(lambda ctrl: self.is_for_training(ctrl), controls))
        
        batch_size = 5
        
        for epoch in range(epochs):
            pairs = self.extract_pairs(controls, neg_samples)
            
            #for batch in split(pairs, 15):
                
                
import gensim
import numpy as np
import string 
from nltk.tokenize import word_tokenize


class Encoder:
    
    def load_glove(self, glove_file):
        with open(glove_file,'r') as f:
            self.glove = {}
            for line in f:
                splitLine = line.split()
                word = splitLine[0]
                embedding = np.array([float(val) for val in splitLine[1:]])
                self.embeddings_dim = embedding.shape[0]
                
                self.glove[word] = embedding
    
    
    def __init__(self, glove_file):
        self.load_glove(glove_file)
        self.char2id = {c: i + 1 for i, c in enumerate(string.printable)}
        
        self.empty = np.zeros(self.embeddings_dim)
        
    
    
    def encode(self, sentences):
        tokenized_sentence = list([word_tokenize(sentence.lower()) for sentence in sentences])
        batch_size = len(sentences)
        
        max_sentence = 0
        max_token = 0
        for sentence in tokenized_sentence:
            max_sentence = max(max_sentence, len(sentence))
            for token in sentence:
                max_token = max(max_token, len(token))
        
        word_embeddings = np.zeros((batch_size, max_sentence, self.embeddings_dim))
        char_ids = np.zeros((batch_size, max_sentence, max_token))
        sentence_lengths = np.zeros(batch_size)
        word_lengths = np.zeros((batch_size, max_sentence))
        
        for i, sentence in enumerate(tokenized_sentence):
            sentence_lengths[i] = len(sentence)
            
            for j, token in enumerate(sentence):
                word_lengths[i, j] = len(token)
                word_embeddings[i, j, :] = self.glove.get(token, self.empty)
                
                for k, char in enumerate(token):
                    cid = self.char2id.get(char, 0)
                    char_ids[i, j, k] = cid
        
        return {
            'word_embeddings': word_embeddings,
            'char_ids': char_ids,
            'sentence_lengths': sentence_lengths,
            'word_lengths': word_lengths
        }
                

SyntaxError: invalid syntax (<ipython-input-32-258dc1dec356>, line 233)

In [30]:
encoder = Encoder('glove.6B/glove.6B.200d.txt')

In [31]:
encoder.encode(['My little ponny', 'Enter your first name, please'])

{'embeddings': array([[[ 0.3038   ,  0.18126  ,  0.46583  , ...,  0.29551  ,
          -0.25326  ,  0.72633  ],
         [ 0.6223   , -0.1426   , -0.16478  , ...,  0.16229  ,
          -0.33931  , -0.060914 ],
         [ 0.       ,  0.       ,  0.       , ...,  0.       ,
           0.       ,  0.       ],
         [ 0.       ,  0.       ,  0.       , ...,  0.       ,
           0.       ,  0.       ],
         [ 0.       ,  0.       ,  0.       , ...,  0.       ,
           0.       ,  0.       ],
         [ 0.       ,  0.       ,  0.       , ...,  0.       ,
           0.       ,  0.       ]],
 
        [[ 0.15807  ,  0.24769  ,  0.20726  , ...,  0.52299  ,
          -0.095942 , -0.0017835],
         [ 0.73603  ,  0.52326  ,  0.65198  , ...,  0.40265  ,
          -0.33904  ,  0.81648  ],
         [-0.14931  ,  0.39057  , -0.44791  , ...,  0.074758 ,
           0.23269  , -0.026688 ],
         [ 0.0055743,  0.24747  ,  0.451    , ...,  0.030439 ,
           0.47877  ,  0.12243  ],
   

In [ ]:
class ControlsModel:
    def __init__(self, a3c_model):
        self.a3c_model = a3c_model
        self.session = a3c_model.session
        self.opt = tf.train.AdamOptimizer(a3c_model.lr)
        
        
    def add_types(self):
        num_controls = len(Types.all_types)

        self.control_types = tf.placeholder(tf.int32, (None), "control_types")
                
        he_init = tf.contrib.layers.variance_scaling_initializer(mode="FAN_AVG")
        xavier_init = tf.contrib.layers.xavier_initializer()
        zero_init = tf.constant_initializer(0)
        
        l2_reg = slim.l2_regularizer(self.a3c_model.l2)
        
        with slim.arg_scope([slim.conv2d, slim.fully_connected],
                              weights_initializer = xavier_init,
                              biases_initializer = zero_init,
                              weights_regularizer = l2_reg
                             ):

            fc2 = slim.fully_connected(self.a3c_model.net, 100, weights_initializer=he_init)
            flat = slim.dropout(fc2, self.a3c_model.dropout, scope='dropout')

            self.types_logits = slim.fully_connected(flat, num_controls, activation_fn=None)
            
            self.types_loss = tf.nn.sparse_softmax_cross_entropy_with_logits(
                labels = self.control_types, 
                logits = self.types_logits)
            
            self.types_loss = tf.reduce_mean(self.types_loss)
            
            self.types_train = self.opt.minimize(self.types_loss)
            
    
    def split(self, arr, batch_size):
        batch = []
        for item in arr:
            if len(batch) >= batch_size:
                yield batch
                batch = []
            
            batch.append(item)
        
        if len(batch) > 0:
            yield batch
            
    
    def batch_as_feed_dict(self, batch, lr, dropout, l2):
        imgs = []
        types = []
        for ctrl in batch:
            img = misc.imread(ctrl['img_file'])
            img = (img - 128.)/128.
            shape = img.shape
            if shape[0] != 224 or shape[1] != 224:
                print('skip, image: {}, control {}'.format(shape, ctrl))
                continue

            imgs.append(img)

            ctrl_type = Types.all_types.index(ctrl['type'])
            types.append(ctrl_type)

        feed_dict = {
            self.a3c_model.img: imgs,
            self.a3c_model.lr: lr,
            self.a3c_model.dropout: dropout,
            self.a3c_model.l2: l2,

            self.control_types: types                
        }
        
        return feed_dict
    
    def train_types(self, controls, batch_size = 10, lr = 0.01, dropout = 0.7, l2 = 0.001, print_loss = True):
        random.shuffle(controls)
        
        batches = self.split(controls, batch_size)
        for batch in batches:
            feed_dict = self.batch_as_feed_dict(batch, lr, dropout, l2)
            
            _, loss = self.session.run([self.types_train, self.types_loss], feed_dict = feed_dict)
            
            if print_loss:
                print(loss)
    
    def measure_types_acc(self, controls):
        batches = self.split(controls, 10)
        correct = 0
        total = 0
        for batch in batches:
            feed_dict = self.batch_as_feed_dict(batch, 0.1, 1., 0.1)
            
            logits = self.session.run(self.types_logits, feed_dict = feed_dict)
            predicted = np.argmax(logits, axis = -1)
            total += len(predicted)
            correct += sum(predicted == feed_dict[self.control_types])
        
        if total == 0:
            return 0.
        
        return float(correct) / total

In [ ]:
tf.reset_default_graph()
session = tf.Session()

a3c = A3CModel(len(Actions.actions), session = session)
a3c.init()

model = ControlsModel(a3c)
model.add_types()

session.run(tf.global_variables_initializer())

In [ ]:
controls = []
with open('dataset.jsonl') as f:
    for line in f:
        ctrl = json.loads(line)
        controls.append(ctrl)

In [ ]:
model.measure_types_acc(controls)

In [ ]:
for i in range(1):
    model.train_types(controls, batch_size = 10, lr = 0.01, dropout = 0.7)
    model.measure_types_acc(controls)
    print('acc: ', model.measure_types_acc(controls))


In [ ]:
stat = {}
max_cnt = 0
for ctrl in controls:
    t = ctrl['type']
    stat[t] = stat.get(t, 0) + 1
    max_cnt = max(max_cnt, stat[t])

print('baseline: ', max_cnt / len(controls))
print('stat: ', stat)


In [ ]:
model.measure_types_acc(controls)